In [10]:
#%pip install transformers[torch]
# %pip install torch
# %pip install datasets
#%pip install accelerate

In [11]:
import torch

DEVICE = "cpu"
# setup optimal acceleration DEVICE 
if torch.backends.mps.is_available():
    DEVICE = torch.device("mps")  # Use Metal Performance Shaders on macOS
elif torch.cuda.is_available():
    DEVICE = torch.device("cuda")  # to check if cuda is an option https://www.restack.io/p/gpu-computing-answer-is-my-gpu-cuda-enabled-cat-ai

print(f"DEVICE is : {DEVICE}")

DEVICE is : cuda


In [12]:
from transformers import AutoModelForMaskedLM, AutoTokenizer

def load_model(model_path_or_id:str, device:str):
    # Replace AutoModelForMaskedLM with the correct class for your task, e.g., AutoModelForSequenceClassification
    model = AutoModelForMaskedLM.from_pretrained(model_path_or_id)
    tokenizer = AutoTokenizer.from_pretrained(model_path_or_id)
    
    model.to(device)

    return model, tokenizer

In [13]:
model_checkpoint = 'distilbert/distilbert-base-uncased'

model, tokenizer = load_model(model_path_or_id=model_checkpoint, device=DEVICE)

In [14]:
#finetuning on IMDb
from datasets import load_dataset
imdb_dataset = load_dataset("imdb")
imdb_dataset["train"][0]

Extracting data files:   0%|          | 0/3 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating unsupervised split:   0%|          | 0/50000 [00:00<?, ? examples/s]

{'text': 'I rented I AM CURIOUS-YELLOW from my video store because of all the controversy that surrounded it when it was first released in 1967. I also heard that at first it was seized by U.S. customs if it ever tried to enter this country, therefore being a fan of films considered "controversial" I really had to see this for myself.<br /><br />The plot is centered around a young Swedish drama student named Lena who wants to learn everything she can about life. In particular she wants to focus her attentions to making some sort of documentary on what the average Swede thought about certain political issues such as the Vietnam War and race issues in the United States. In between asking politicians and ordinary denizens of Stockholm about their opinions on politics, she has sex with her drama teacher, classmates, and married men.<br /><br />What kills me about I AM CURIOUS-YELLOW is that 40 years ago, this was considered pornographic. Really, the sex and nudity scenes are few and far be

In [15]:
def tokenize_function(examples):
    result = tokenizer(examples["text"])
    if tokenizer.is_fast:
        result["word_ids"] = [result.word_ids(i) for i in range(len(result["input_ids"]))]
    return result


# Use batched=True to activate fast multithreading!
tokenized_datasets = imdb_dataset.map(
    tokenize_function, batched=True, remove_columns=["text", "label"]
)
tokenized_datasets

Map:   0%|          | 0/25000 [00:00<?, ? examples/s]

Token indices sequence length is longer than the specified maximum sequence length for this model (720 > 512). Running this sequence through the model will result in indexing errors


Map:   0%|          | 0/25000 [00:00<?, ? examples/s]

Map:   0%|          | 0/50000 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask', 'word_ids'],
        num_rows: 25000
    })
    test: Dataset({
        features: ['input_ids', 'attention_mask', 'word_ids'],
        num_rows: 25000
    })
    unsupervised: Dataset({
        features: ['input_ids', 'attention_mask', 'word_ids'],
        num_rows: 50000
    })
})

In [16]:
chunk_size = tokenizer.model_max_length // 2
def group_texts(examples):
    # Concatenate all texts
    concatenated_examples = {k: sum(examples[k], []) if isinstance(examples[k][0], list) else examples[k] for k in examples.keys()}
    # Compute the total length of concatenated texts
    total_length = len(concatenated_examples[list(examples.keys())[0]])
    # Drop the last chunk if it's smaller than chunk_size
    total_length = (total_length // chunk_size) * chunk_size
    # Split by chunks of chunk_size
    result = {
        k: [t[i : i + chunk_size] for i in range(0, total_length, chunk_size)]
        for k, t in concatenated_examples.items()
    }
    # Create a new "labels" column that is a copy of "input_ids"
    result["labels"] = result["input_ids"].copy()
    return result


In [17]:
lm_datasets = tokenized_datasets.map(group_texts, batched=True)
lm_datasets

Map:   0%|          | 0/25000 [00:00<?, ? examples/s]

Map:   0%|          | 0/25000 [00:00<?, ? examples/s]

Map:   0%|          | 0/50000 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask', 'word_ids', 'labels'],
        num_rows: 30639
    })
    test: Dataset({
        features: ['input_ids', 'attention_mask', 'word_ids', 'labels'],
        num_rows: 29946
    })
    unsupervised: Dataset({
        features: ['input_ids', 'attention_mask', 'word_ids', 'labels'],
        num_rows: 61465
    })
})

In [18]:
from transformers import DataCollatorForLanguageModeling, TrainingArguments, Trainer

tokenizer.pad_token = "[PAD]" #tokenizer.eos_token
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm_probability=0.15)

In [20]:
from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(
    output_dir="movie_model",
    save_strategy="epoch",
    learning_rate=2e-5,
    num_train_epochs=1,
    weight_decay=0.01,
    push_to_hub=False,
    per_device_train_batch_size=32,  # lower this in case of memory problems
    fp16 = True
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=lm_datasets["train"],
    eval_dataset=lm_datasets["test"],
    data_collator=data_collator,
    tokenizer=tokenizer,
)

%time trainer.train()

  0%|          | 0/958 [00:00<?, ?it/s]

{'loss': 2.5117, 'grad_norm': 3.6085495948791504, 'learning_rate': 9.624217118997912e-06, 'epoch': 0.52}
{'train_runtime': 314.4712, 'train_samples_per_second': 97.43, 'train_steps_per_second': 3.046, 'train_loss': 2.473961943624413, 'epoch': 1.0}


TrainOutput(global_step=958, training_loss=2.473961943624413, metrics={'train_runtime': 314.4712, 'train_samples_per_second': 97.43, 'train_steps_per_second': 3.046, 'total_flos': 2030770630812672.0, 'train_loss': 2.473961943624413, 'epoch': 1.0})

In [21]:
def top_k_prediction(masked_text, model, tokenizer, k=10):
    model.to(DEVICE) 
    inputs = tokenizer(masked_text, return_tensors="pt").to(DEVICE)
    logits = model(**inputs).logits
    mask_token_index = torch.where(inputs["input_ids"]==tokenizer.mask_token_id)[1]
    mask_token_logits = logits[0, mask_token_index, :]
    return [tokenizer.decode(t) for t in torch.topk(mask_token_logits, k, dim=1).indices[0].tolist()]

In [30]:
k = 30
#origional, number the output
list(zip(range(k), top_k_prediction("The [MASK] outside is great", model, tokenizer, k)))


[(0, 'weather'),
 (1, 'view'),
 (2, 'world'),
 (3, 'time'),
 (4, 'air'),
 (5, 'feeling'),
 (6, 'space'),
 (7, 'climate'),
 (8, 'sun'),
 (9, 'scenery'),
 (10, 'sea'),
 (11, 'temperature'),
 (12, 'look'),
 (13, 'life'),
 (14, 'atmosphere'),
 (15, 'wind'),
 (16, 'the'),
 (17, 'area'),
 (18, 'freedom'),
 (19, 'environment'),
 (20, 'way'),
 (21, 'sky'),
 (22, 'music'),
 (23, 'heat'),
 (24, 'land'),
 (25, 'far'),
 (26, 'art'),
 (27, 'beauty'),
 (28, 'silence'),
 (29, 'color')]

In [31]:
tuned_model, tokenizer = load_model(model_path_or_id="movie_model\checkpoint-958", device=DEVICE)
list(zip(range(k), top_k_prediction("The [MASK] outside is great", tuned_model, tokenizer, k)))

[(0, 'weather'),
 (1, 'view'),
 (2, 'scenery'),
 (3, 'air'),
 (4, 'atmosphere'),
 (5, 'world'),
 (6, 'space'),
 (7, 'sea'),
 (8, 'look'),
 (9, 'climate'),
 (10, 'time'),
 (11, 'feeling'),
 (12, 'temperature'),
 (13, 'life'),
 (14, 'sun'),
 (15, 'landscape'),
 (16, 'sky'),
 (17, 'land'),
 (18, 'area'),
 (19, 'music'),
 (20, 'light'),
 (21, 'sight'),
 (22, 'environment'),
 (23, 'the'),
 (24, 'beauty'),
 (25, 'art'),
 (26, 'silence'),
 (27, 'city'),
 (28, 'garden'),
 (29, 'lighting')]

# TODO
Implement some measures between the returned logits, distributions or topk results from models. 

In [1]:
#perplexity of target words

In [2]:
#recall (how many words stayed the same)

In [ ]:
#jaccard Similarity: Measures overlap between two sets or lists.

In [ ]:
#edit distance
